In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv(r"C:\Users\anmol\Documents\Data Science with Python(STP)\bingebuddy project\project_dataset\Omniflix_cleaned.csv")
df

,content_id,content_title,content_type,release_year,content_genres,duration,avg_rating,content_description,director/studio_for_anime,cast/producer_for_anime,year_bucket,Final_Content
0,32281,Kimi no Na wa.,Movie,2016.0,"Supernatural, Drama, Romance, School",1 hr. 46 min.,9.19,PG-13 - Teens 13 or older\nKimi no Na wa. is a...,CoMix Wave Films,"Kadokawa Shoten, Toho, Sound Team Don Juan, La...",2010s,pg-13 - teens 13 or older\nkimi no na wa. is a...
1,5114,Fullmetal Alchemist: Brotherhood,TV,2009.0,"Action, Military, Adventure, Comedy, Drama, Ma...",24 min. per ep.,9.25,R - 17+ (violence & profanity)\nFullmetal Alch...,Bones,"Aniplex, Square Enix, Mainichi Broadcasting Sy...",2000s,r - 17+ (violence & profanity)\nfullmetal alch...
2,28977,Gintama°,TV,2015.0,"Action, Comedy, Historical, Parody, Samurai, S...",24 min. per ep.,9.15,R - 17+ (violence & profanity)\nGintama° is a ...,Bandai Namco Pictures,"TV Tokyo, Aniplex, Dentsu",2010s,r - 17+ (violence & profanity)\ngintama° is a ...
3,9253,Steins;Gate,TV,2011.0,"Thriller, Sci-Fi",24 min. per ep.,9.14,PG-13 - Teens 13 or older\nSteins;Gate is a TV...,White Fox,"Frontier Works, Media Factory, Movic, AT-X, Ka...",2010s,pg-13 - teens 13 or older\nsteins;gate is a tv...
4,9969,Gintama&#039;,TV,2011.0,"Action, Sci-Fi, Comedy, Historical, Parody, Sa...",24 min. per ep.,9.11,PG-13 - Teens 13 or older\nGintama&#039; is a ...,Sunrise,"TV Tokyo, Aniplex, Dentsu, Trinity Sound, Mira...",2010s,pg-13 - teens 13 or older\ngintama&#039; is a ...
...,...,...,...,...,...,...,...,...,...,...,...,...
65787,s8803,Zodiac,Movie,2007.0,"Cult Movies, Dramas, Thrillers",158 min,7.70,"R\nA political cartoonist, a crime reporter an...",David Fincher,"Mark Ruffalo, Jake Gyllenhaal, Robert Downey J...",2000s,"r\na political cartoonist, a crime reporter an..."
65788,s8804,Zombie Dumb,TV Show,2018.0,"Kids' TV, Korean TV Shows, TV Comedies",2 Seasons,5.00,"TV-Y7\nWhile living alone in a spooky town, a ...",Unknown,Unknown,2010s,"tv-y7\nwhile living alone in a spooky town, a ..."
65789,s8805,Zombieland,Movie,2009.0,"Comedies, Horror Movies",88 min,7.60,R\nLooking to survive in a world taken over by...,Ruben Fleischer,"Jesse Eisenberg, Woody Harrelson, Emma Stone, ...",2000s,r\nlooking to survive in a world taken over by...
65790,s8806,Zoom,Movie,2006.0,"Children & Family Movies, Comedies",88 min,4.40,"PG\nDragged from civilian life, a former super...",Peter Hewitt,"Tim Allen, Courteney Cox, Chevy Chase, Kate Ma...",2000s,"pg\ndragged from civilian life, a former super..."


In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# --- Step 1: TF-IDF Vectorization ---
tfidf = TfidfVectorizer(stop_words='english', max_features=50000)
tfidf_matrix = tfidf.fit_transform(df["Final_Content"])

# --- Step 2: Dimensionality Reduction (optional but helps memory) ---
svd = TruncatedSVD(n_components=200, random_state=42)
tfidf_reduced = svd.fit_transform(tfidf_matrix)

print("Reduced TF-IDF shape:", tfidf_reduced.shape)

Reduced TF-IDF shape: (65792, 200)


In [4]:
from sklearn.preprocessing import MinMaxScaler

# Normalize ratings (0-1 range)
scaler = MinMaxScaler()
df["avg_rating"] = scaler.fit_transform(df[["avg_rating"]])

In [5]:
# --- Step 3: Function to get recommendations without full similarity matrix ---
def get_recommendations(title, topN=10, alpha=0.8):
    """
    alpha = weight for similarity (0-1)
    (1-alpha) = weight for rating
    """
    # Find index of the movie/show
    idx = df[df["content_title"] == title].index[0]

    # Get the vector of the target moviea
    target_vec = tfidf_reduced[idx].reshape(1, -1)

    # Compute cosine similarity
    sim_scores = cosine_similarity(target_vec, tfidf_reduced).flatten()

    # Compute hybrid score
    hybrid_scores = alpha * sim_scores + (1 - alpha) * df["avg_rating"].values

    # Sort scores (excluding itself)
    sim_indices = np.argsort(hybrid_scores)[::-1]
    sim_indices = [i for i in sim_indices if i != idx][:topN]

    # Build DataFrame with details
    results = df.loc[sim_indices, ["content_title", "content_genres", "content_type", "avg_rating"]].copy()
    results["similarity_score"] = sim_scores[sim_indices]
    results["hybrid_score"] = hybrid_scores[sim_indices]

    results = results.sort_values(by="hybrid_score", ascending=False).reset_index(drop=True)
    return results

In [6]:
recommendations = get_recommendations("Squid Game", topN=40, alpha = 0.8)
pd.DataFrame(recommendations)

,content_title,content_genres,content_type,avg_rating,similarity_score,hybrid_score
0,Move to Heaven,"International TV Shows, TV Dramas",TV Show,0.86,0.942388,0.925911
1,Itaewon Class,"International TV Shows, Korean TV Shows, TV Dr...",TV Show,0.82,0.920462,0.900370
2,Law School,"Crime TV Shows, International TV Shows, TV Dramas",TV Show,0.81,0.909765,0.889812
3,Prison Playbook,"International TV Shows, Korean TV Shows, TV Co...",TV Show,0.85,0.899537,0.889629
4,Mystic Pop-up Bar,"International TV Shows, Korean TV Shows, TV Co...",TV Show,0.80,0.908339,0.886672
5,Designated Survivor: 60 Days,"International TV Shows, Korean TV Shows, TV Dr...",TV Show,0.82,0.891059,0.876847
6,Chief of Staff,"International TV Shows, Korean TV Shows, TV Dr...",TV Show,0.79,0.884379,0.865503
7,Misaeng,"International TV Shows, Korean TV Shows, TV Dr...",TV Show,0.86,0.866456,0.865164
8,Run On,"International TV Shows, Romantic TV Shows, TV ...",TV Show,0.76,0.888248,0.862598
9,It's Okay to Not Be Okay,"International TV Shows, Korean TV Shows, Roman...",TV Show,0.87,0.856862,0.859490


In [8]:
import joblib

In [11]:
# saving verctorizer and svd
joblib.dump(tfidf, 'tfidf_vectorizer.pkl')
joblib.dump(svd, 'svd_model.pkl')

# saving reduced features in numpy array
np.save('tfidf_reduced.npy', tfidf_reduced)

# saving dataframe  
df.to_pickle('content_metadata.pkl')